In [3]:
from mindquantum.utils import random_circuit
from mindquantum.core.circuit import Circuit
from huawei_graph_tool import ConvertCircuit,my_random_circuit

import random,os,torch
import networkx as nx
from mindquantum.io.qasm import HiQASM
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import RandomLinkSplit
# from Pytorch_VGAE import CustomGraphDataset,VGAEModel
import pandas as pd
import torch.nn.functional as F
from torch_geometric.utils import from_scipy_sparse_matrix, dense_to_sparse

In [2]:
def test_vgae_model(model, A, X):
    # 确保模型处于评估模式
    model.eval()
    
    # 将输入数据转换为张量
    A = sp.coo_matrix(A)  # 确保 A 是 COO 格式
    edge_index, _ = from_scipy_sparse_matrix(A)  # 转换为边索引
    X = torch.tensor(X, dtype=torch.float)  # 特征矩阵转换为张量

    # 前向传播：编码
    with torch.no_grad():
        z = model.encode(X, edge_index)  # 编码得到潜在表示 Z
        
    # 假设我们使用模型的解码器来重建边
    # 这里需要实现重建的逻辑，假设你有一个解码函数
    # 此处的 decoder 需要根据你的模型定义进行实现
    # 示例代码:
    reconstructed_edge_index = model.decoder(z)  # 需要实现解码逻辑

    # 计算重建误差
    # 将重建的边索引转换为稀疏矩阵
    reconstructed_adj = dense_to_sparse(reconstructed_edge_index)

    # 计算重建误差（如 MSE 或其他）
    loss = F.mse_loss(reconstructed_adj, A)  # 使用合适的损失函数

    print(f'Reconstruction loss: {loss.item():.4f}')

# 数据集实例化
root = './VGAE_dataset'
dataset = CustomGraphDataset(root=root)
# 加载模型
model = VGAEModel(in_channels=dataset[0].x.size(1), out_channels=16)
model.load_state_dict(torch.load('vgae_model.pth'))
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))  # 如果有 GPU 可用，使用它

# 示例用法
# 假设你有一个邻接矩阵 A 和特征矩阵 X
A_test = pd.read_csv('path_to_A.csv', header=None).values  # 读取测试数据
X_test = pd.read_csv('path_to_X.csv', header=None).values  # 读取测试数据

# 测试模型
test_vgae_model(model, A_test, X_test)


NameError: name 'CustomGraphDataset' is not defined